In [1]:
# Import required packages
import pandas as pd
import numpy as np

In [2]:
# Variables to tweak for data creation
sample_size = 10

In [3]:
# Read data into environment
last_names = pd.read_csv('data/common_surnames_census_2000.csv')
first_names = pd.read_csv('data/ssa_names_db.csv')

In [4]:
# Data preprocessing for Last Names

# Replace unknown value with 0
# Prevents conflicts when finding max(percentages)
last_names2 = last_names.replace('(S)', 0.00)

# Convert percentage columns from strings to floats
for column in last_names2.columns[1:]:
    if last_names2[column].dtype == 'object':
        last_names2[column] = last_names2[column].astype(float)

# Create new column based on the ethnicity label with highest probability
last_names2['predominant'] = last_names2.iloc[:,5:].idxmax(1).str.replace('pct', '')

# Sample evenly through each unique dominant ethnicity
# Prevents most names being white and promotes even representation
last_names_final = last_names2.groupby('predominant').apply(lambda ethnicity: ethnicity.sample(sample_size)).reset_index(drop=True)